In [64]:
import time, copy
import numpy as np

def load_p1():
  weights = np.array([23, 31, 29, 44, 53, 38, 69, 85, 89, 8])
  profits = np.array([92, 57, 49, 68, 60, 43, 67, 84, 87, 72])
  capacity = 165
  return weights, profits, capacity

def load_p8():
  weights = [382745, 799601, 909247, 729069, 467902, 44328, 34610, 698150, 823460, 903959, 853665, 551830, 610856, 670702, 488960, 951111, 323046, 446298, 931161, 31385, 496951, 264724, 224916, 169684]
  profits = [825594, 1677009, 1676628, 1523970, 943972, 97426, 69666, 1296457, 1679693, 1902996, 1844992, 1049289, 1252836, 1319836, 953277, 2067538, 675367, 853655, 1826027, 65731, 901489, 577243, 466257, 369261]
  capacity = 6404180
  return weights, profits, capacity

def load_p2_8():
  weights = [983, 982, 981, 980, 979, 978, 488, 976, 972, 486, 486, 972, 972, 485, 485, 969, 966, 483, 964, 963, 961, 958, 959]
  profits = [81, 980, 979, 978, 977, 976, 487, 974, 970, 485, 485, 970, 970, 484, 484, 976, 974, 482, 962, 961, 959, 958, 857]
  capacity = 10000
  return weights, profits, capacity

def load_p2_10():
  weights = [84, 83, 43, 4, 44, 6, 82, 92, 25, 83, 56, 18, 58, 14, 48, 70, 96, 32, 68, 92]
  profits = [91, 72, 90, 46, 55, 8, 35, 75, 61, 15, 77, 40, 63, 75, 29, 75, 17, 78, 40, 44]
  capacity = 879
  return weights, profits, capacity

def get_data(data_name):#Ricky: 取資料 不管
  if data_name == "p1":
    return load_p1()
  elif data_name == "p8":
    return load_p8()
  elif data_name == "p2_8":
    return load_p2_8()
  elif data_name == "p2_10":
    return load_p2_10()

def selection(population, fitness, k=3): #Ricky: 隨機挑選選擇的 0101101000, (幾個群體, 群體個數) fitness 隨機取得的重量與價值結果
  parents = np.empty((population.shape[0], population.shape[1])) #Ricky: 用原本隨機挑選的群體大小與群體個數作為父代
  for parent_num in range(population.shape[0]): #Ricky: for loop 群體大小 = 幾個群體
    selection_ix = np.random.randint(len(population)) #Ricky: 隨機選一個基因 隨機一個index
    for ix in np.random.randint(0, len(population), k-1): #Ricky: 隨機選一個二維的
      if fitness[ix] > fitness[selection_ix]:
        selection_ix = ix
    parents[parent_num, :] = population[selection_ix, :]
  return parents

def crossover(parents, pc=0.8):
  dim_size = parents.shape[1]
  offspring = copy.copy(parents)
  for i in range(int(parents.shape[0] / 2)):
    if np.random.random() > pc:
      x, y = 2*i, 2*i+1
      k = np.random.randint(dim_size) # crossover point
      offspring[x][k+1:] = parents[y][k+1:]
      offspring[y][k+1:] = parents[x][k+1:]
  return offspring

def mutation(offspring, pm=0.05):
  d_list = np.array([i for i in range(offspring.shape[1])])
  for i in range(offspring.shape[0]):
    if np.random.random() < pm:
      mutation_point = np.random.choice(d_list, 2, replace=False)
      for j in mutation_point:
        if offspring[i][j] == 1:
          offspring[i][j] = 0
        else:
          offspring[i][j] = 1
  return offspring

def get_fitness(X, w, p, b):#Ricky: X 隨機挑選選擇的 w 全部重量的數值 p 全部價值的數值 b 能裝的重量
  results = np.zeros(population_size)#Ricky: return 有population_size個大小的list 裡面都放 0
  for i in range(X.shape[0]):#Ricky: .shape[0] = 矩陣行數 = 幾個群體
    totalWeight = totalProfit = 0 #Ricky: 都先為0
    for j in range(X.shape[1]):#Ricky: .shape[1] = 矩陣列數 = 群體個數
      if totalWeight + w[j] <= b: #Ricky: 假如背包還可以裝
        totalWeight += X[i][j] * w[j] #Ricky: 重量加總 = 有沒有選到0/1 * 該重量
        totalProfit += X[i][j] * p[j] #Ricky: 價值加總 = 有沒有選到0/1 * 該價值
    results[i] = totalProfit
  return results

start_time = time.time()
np.random.seed(7)

datasets = ["p1", "p8", "p2_8", "p2_10"]
result = []
for data_name in datasets:
  weights, profits, capacity = get_data(data_name)
  dimension_size = len(weights) #Ricky: 群體個數 -> 重量的地方有幾個數值 EX1:10個
  population_size = dimension_size*3 #Ricky: 利用浮動改變大小
  iteration_size = dimension_size//5 #Ricky: 利用浮動改變大小 原因: 不是每一個set都需要這麼多次才能得到最佳解 --------------------------------------------------///<<<< SOL >>>>
  pc = 0.8
  pm = 0.05
  best_solution, best_fitness = [], []

  population = np.random.randint(2, size=(population_size, dimension_size)) #Ricky: 隨機挑選選擇的 0101101000, (幾個群體, 群體個數)
  fitness = np.zeros(population_size) #Ricky: return 有population_size個大小的list 裡面都放 0
  fitness = get_fitness(population, weights, profits, capacity)  #Ricky: 隨機取得的重量與價值結果
  for iter in range(iteration_size): #Ricky: iteration_size要繁衍幾代
    parents = selection(population, fitness)
    offspring_crossover = crossover(parents, pc)
    offspring_mutation = mutation(offspring_crossover, pm)
    population[:] = offspring_mutation[:]
    fitness = get_fitness(population, weights, profits, capacity)
  
  result.append(np.max(fitness))
print(result)
print(np.mean(result))
end_time = time.time()
print("Spend %0.4f sec(s)" % (end_time - start_time))

[349.0, 13518963.0, 9750.0, 887.0]
3382487.25
Spend 0.0491 sec(s)


3341592.0
Spend 0.0480 sec(s)
